In [2]:
from ultralytics import YOLO

# 1. Model ve Veri Yolu
dataset_path = r"C:\Users\User\Desktop\fabrika\dataset_pano"
model = YOLO('yolov8n-cls.pt') # Nano model (hızlı ve hafif)

if __name__ == '__main__':
    # 2. Gelişmiş Eğitim Ayarları
    results = model.train(
        data=dataset_path,
        epochs=50,          # Maksimum 100 tur (Early stopping bunu kesecektir)
        imgsz=224,
        batch=16,
        
        # --- EARLY STOPPING & CHECKPOINT ---
        patience=6,         # 10 epoch boyunca gelişme olmazsa eğitimi DURDUR
        save=True,           # Checkpoint'leri otomatik kaydet (best.pt, last.pt)
        save_period=5,       # Her 5 epoch'ta bir yedek model kaydet (isteğe bağlı)
        
        # --- OPTİMİZASYON ---
        optimizer='AdamW',   # Daha kararlı bir öğrenme için
        lr0=0.001,           # Başlangıç öğrenme oranı
        name='pano_denetim_v1'
    )

    print("\n✅ Eğitim Tamamlandı!")
    print("En iyi model şurada: runs/classify/pano_denetim_v1/weights/best.pt")

New https://pypi.org/project/ultralytics/8.4.8 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.217  Python-3.13.7 torch-2.9.0+cpu CPU (AMD Ryzen 7 4800H with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\User\Desktop\fabrika\dataset_pano, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pano_den

In [7]:
import cv2
import numpy as np
from ultralytics import YOLO

# 1. Modeli ve Videoyu Tanımla
MODEL_PATH = r"C:\Users\User\runs\classify\pano_denetim_v12\weights\best.pt"
cap = cv2.VideoCapture(r"test.mp4")

# Çıktı Videosu Ayarları
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('denetim_sonucu.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

model = YOLO(MODEL_PATH)
PANO_PTS = np.float32([[200, 620], [420, 660], [430, 780], [220, 750]])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # Perspektif Kırpma
    dst_pts = np.float32([[0, 0], [224, 0], [224, 224], [0, 224]])
    matrix = cv2.getPerspectiveTransform(PANO_PTS, dst_pts)
    pano_crop = cv2.warpPerspective(frame, matrix, (224, 224))
    
    # Tahmin
    results = model.predict(pano_crop, verbose=False)[0]
    label = results.names[results.probs.top1]
    conf = results.probs.top1conf.item()

    # Görselleştirme
    color = (0, 255, 0) if label == "kapali" else (0, 0, 255)
    cv2.polylines(frame, [PANO_PTS.astype(np.int32)], True, color, 3)
    cv2.putText(frame, f"{label} %{conf:.2f}", (int(PANO_PTS[0][0]), int(PANO_PTS[0][1]-10)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Videoya yaz
    out.write(frame)

cap.release()
out.release()
print("✅ Analiz tamamlandı! 'denetim_sonucu.mp4' dosyasını kontrol edebilirsin.")

✅ Analiz tamamlandı! 'denetim_sonucu.mp4' dosyasını kontrol edebilirsin.


In [8]:
import cv2
import numpy as np
from ultralytics import YOLO

# 1. Modeli ve Videoyu Tanımla
MODEL_PATH = r"C:\Users\User\runs\classify\pano_denetim_v12\weights\best.pt"
cap = cv2.VideoCapture(r"test2.mp4")

# Çıktı Videosu Ayarları
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('denetim_sonucu2.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

model = YOLO(MODEL_PATH)
PANO_PTS = np.float32([[200, 620], [420, 660], [430, 780], [220, 750]])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # Perspektif Kırpma
    dst_pts = np.float32([[0, 0], [224, 0], [224, 224], [0, 224]])
    matrix = cv2.getPerspectiveTransform(PANO_PTS, dst_pts)
    pano_crop = cv2.warpPerspective(frame, matrix, (224, 224))
    
    # Tahmin
    results = model.predict(pano_crop, verbose=False)[0]
    label = results.names[results.probs.top1]
    conf = results.probs.top1conf.item()

    # Görselleştirme
    color = (0, 255, 0) if label == "kapali" else (0, 0, 255)
    cv2.polylines(frame, [PANO_PTS.astype(np.int32)], True, color, 3)
    cv2.putText(frame, f"{label} %{conf:.2f}", (int(PANO_PTS[0][0]), int(PANO_PTS[0][1]-10)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Videoya yaz
    out.write(frame)

cap.release()
out.release()
print("✅ Analiz tamamlandı! 'denetim_sonucu.mp4' dosyasını kontrol edebilirsin.")

✅ Analiz tamamlandı! 'denetim_sonucu.mp4' dosyasını kontrol edebilirsin.


In [9]:
import shutil
import os

# Kaynak ve hedef yollar
kaynak = r"C:\Users\User\runs\classify\pano_denetim_v12\weights\best.pt"
masaustu = os.path.join(os.path.expanduser("~"), "Desktop", "fabrika_pano_modeli.pt")

try:
    shutil.copy(kaynak, masaustu)
    print(f"✅ Model başarıyla kopyalandı: {masaustu}")
except FileNotFoundError:
    print("❌ Dosya bulunamadı, lütfen yolu kontrol et.")

✅ Model başarıyla kopyalandı: C:\Users\User\Desktop\fabrika_pano_modeli.pt
